In [1]:
import os
import sys
import numpy as np
import pandas as pd
import collections
import itertools
from mirnylib import h5dict, genome
from hiclib import mapping, fragmentHiC
import cooler
from multiprocessing import Pool

hi
hello from new mapping


In [2]:
genome_db = genome.Genome('/home/zzeng/cloud_research/PengGroup/ZZeng/Data/Haihui/Tcf1/HiC-seq/Jun2016/exp/iterative_mapping/genome.fa')

In [3]:
import HTSeq
def iv_to_str(iv):
        return iv.chrom + ':' + str(iv.start) + '-' + str(iv.end)  
    
idx = pd.IndexSlice

In [4]:
peaks_dict = {}

Tcf1_peaks_bed_file = HTSeq.BED_Reader('processed/HiC_around_Tcf1_peaks/WT_CD8_Tcf1_peaks.bed')
for alt in Tcf1_peaks_bed_file:
    peaks_dict[iv_to_str(alt.iv)] = alt.iv

In [5]:
up_genes_dict = {}

up_genes_bed_file = HTSeq.BED_Reader('processed/HiC_around_Tcf1_peaks/diff_genes/HP_WT_up_only_gene_promoter_iv.bed')
for alt in up_genes_bed_file:
    up_genes_dict[alt.name] = alt.iv

In [6]:
down_genes_dict = {}

down_genes_bed_file = HTSeq.BED_Reader('processed/HiC_around_Tcf1_peaks/diff_genes/HP_WT_down_only_gene_promoter_iv.bed')
for alt in down_genes_bed_file:
    down_genes_dict[alt.name] = alt.iv

In [7]:
os.chdir('/home/zzeng/cloud_research/PengGroup/ZZeng/Data/Haihui/Tcf1/HiC-seq/Combine_Jun_Jul_2016/iterative_mapping_3')

import numpy as np
import pandas as pd
import collections
import cooler

cutoff = 10e-5
sig_df = pd.read_csv('fit-hi-c/WT_CD8.spline_pass2.significances.txt.gz', header=0, sep='\t')
sig_df = sig_df[sig_df['q-value'] < cutoff]

mycooler = cooler.Cooler('WT_CD8-res-10K.cool')
bin_df = mycooler.bins()[:]
bin_df['mid'] = ((bin_df['start'] + bin_df['end']) / 2).astype(int)

bin_start_dict = collections.defaultdict(lambda: {})
for chrom in mycooler.chromnames:
    bin_start_dict[chrom] = dict(bin_df.loc[bin_df.chrom == chrom, ['mid', 'start']].values)

bin_end_dict = collections.defaultdict(lambda: {})
for chrom in mycooler.chromnames:
    bin_end_dict[chrom] = dict(bin_df.loc[bin_df.chrom == chrom, ['mid', 'end']].values)

sig_df['start1'] = sig_df.apply(lambda row: bin_start_dict[row['chr1']][row['fragmentMid1']], axis=1)
sig_df['start2'] = sig_df.apply(lambda row: bin_start_dict[row['chr2']][row['fragmentMid2']], axis=1)

sig_df['name'] = sig_df.index
sig_df['strand'] = '.'
sig_df['score'] = -np.log10(sig_df['p-value'])
sig_df.loc[sig_df.score == np.inf, 'score'] = 1000

chr_matrix_list = []
for chrom in mycooler.chromnames:
    chr_matrix = mycooler.matrix(balance=True, as_pixels=True, join=True).fetch(chrom).fillna(0)
    chr_matrix = chr_matrix[(abs(chr_matrix.start1 - chr_matrix.start2) >= 20000) & (abs(chr_matrix.start1 - chr_matrix.start2) <= 2000000)]
    chr_matrix_list.append(chr_matrix)

matrix_df = pd.concat(chr_matrix_list).set_index(['chrom1', 'start1', 'start2'])
sig_df['balanced_count'] = sig_df.apply(lambda row: matrix_df.loc[idx[row['chr1'], row['start1'], row['start2']], 'balanced'], axis=1)

WT_sig_df = sig_df

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log10


In [8]:
os.chdir('/home/zzeng/cloud_research/PengGroup/ZZeng/Data/Haihui/Tcf1/HiC-seq/Combine_Jun_Jul_2016/iterative_mapping_3')

import numpy as np
import pandas as pd
import collections
import cooler

cutoff = 10e-5
sig_df = pd.read_csv('fit-hi-c/DKO_CD8.spline_pass2.significances.txt.gz', header=0, sep='\t')
sig_df = sig_df[sig_df['q-value'] < cutoff]

mycooler = cooler.Cooler('DKO_CD8-res-10K.cool')
bin_df = mycooler.bins()[:]
bin_df['mid'] = ((bin_df['start'] + bin_df['end']) / 2).astype(int)

bin_start_dict = collections.defaultdict(lambda: {})
for chrom in mycooler.chromnames:
    bin_start_dict[chrom] = dict(bin_df.loc[bin_df.chrom == chrom, ['mid', 'start']].values)

bin_end_dict = collections.defaultdict(lambda: {})
for chrom in mycooler.chromnames:
    bin_end_dict[chrom] = dict(bin_df.loc[bin_df.chrom == chrom, ['mid', 'end']].values)

sig_df['start1'] = sig_df.apply(lambda row: bin_start_dict[row['chr1']][row['fragmentMid1']], axis=1)
sig_df['start2'] = sig_df.apply(lambda row: bin_start_dict[row['chr2']][row['fragmentMid2']], axis=1)

sig_df['name'] = sig_df.index
sig_df['strand'] = '.'
sig_df['score'] = -np.log10(sig_df['p-value'])
sig_df.loc[sig_df.score == np.inf, 'score'] = 1000

chr_matrix_list = []
for chrom in mycooler.chromnames:
    chr_matrix = mycooler.matrix(balance=True, as_pixels=True, join=True).fetch(chrom).fillna(0)
    chr_matrix = chr_matrix[(abs(chr_matrix.start1 - chr_matrix.start2) >= 20000) & (abs(chr_matrix.start1 - chr_matrix.start2) <= 2000000)]
    chr_matrix_list.append(chr_matrix)

matrix_df = pd.concat(chr_matrix_list).set_index(['chrom1', 'start1', 'start2'])
sig_df['balanced_count'] = sig_df.apply(lambda row: matrix_df.loc[idx[row['chr1'], row['start1'], row['start2']], 'balanced'], axis=1)

DKO_sig_df = sig_df

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log10


In [10]:
os.chdir('/home/zzeng/cloud_research/PengGroup/ZZeng/Data/Haihui/Tcf1/HiC-seq/Jul2017/iterative_mapping')

import numpy as np
import pandas as pd
import collections
import cooler

cutoff = 10e-5
sig_df = pd.read_csv('fit-hi-c/stim_WT_CD8.spline_pass2.significances.txt.gz', header=0, sep='\t')
sig_df = sig_df[sig_df['q-value'] < cutoff]

mycooler = cooler.Cooler('stim_WT_CD8-res-10K.cool')
bin_df = mycooler.bins()[:]
bin_df['mid'] = ((bin_df['start'] + bin_df['end']) / 2).astype(int)

bin_start_dict = collections.defaultdict(lambda: {})
for chrom in mycooler.chromnames:
    bin_start_dict[chrom] = dict(bin_df.loc[bin_df.chrom == chrom, ['mid', 'start']].values)

bin_end_dict = collections.defaultdict(lambda: {})
for chrom in mycooler.chromnames:
    bin_end_dict[chrom] = dict(bin_df.loc[bin_df.chrom == chrom, ['mid', 'end']].values)

sig_df['start1'] = sig_df.apply(lambda row: bin_start_dict[row['chr1']][row['fragmentMid1']], axis=1)
sig_df['start2'] = sig_df.apply(lambda row: bin_start_dict[row['chr2']][row['fragmentMid2']], axis=1)

sig_df['name'] = sig_df.index
sig_df['strand'] = '.'
sig_df['score'] = -np.log10(sig_df['p-value'])
sig_df.loc[sig_df.score == np.inf, 'score'] = 1000

chr_matrix_list = []
for chrom in mycooler.chromnames:
    chr_matrix = mycooler.matrix(balance=True, as_pixels=True, join=True).fetch(chrom).fillna(0)
    chr_matrix = chr_matrix[(abs(chr_matrix.start1 - chr_matrix.start2) >= 20000) & (abs(chr_matrix.start1 - chr_matrix.start2) <= 2000000)]
    chr_matrix_list.append(chr_matrix)

matrix_df = pd.concat(chr_matrix_list).set_index(['chrom1', 'start1', 'start2'])
sig_df['balanced_count'] = sig_df.apply(lambda row: matrix_df.loc[idx[row['chr1'], row['start1'], row['start2']], 'balanced'], axis=1)

stim_WT_sig_df = sig_df

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log10


In [11]:
os.chdir('/home/zzeng/cloud_research/PengGroup/ZZeng/Data/Haihui/Tcf1/HiC-seq/Jul2017/iterative_mapping')


import numpy as np
import pandas as pd
import collections
import cooler

cutoff = 10e-5
sig_df = pd.read_csv('fit-hi-c/stim_DKO_CD8.spline_pass2.significances.txt.gz', header=0, sep='\t')
sig_df = sig_df[sig_df['q-value'] < cutoff]

mycooler = cooler.Cooler('stim_DKO_CD8-res-10K.cool')
bin_df = mycooler.bins()[:]
bin_df['mid'] = ((bin_df['start'] + bin_df['end']) / 2).astype(int)

bin_start_dict = collections.defaultdict(lambda: {})
for chrom in mycooler.chromnames:
    bin_start_dict[chrom] = dict(bin_df.loc[bin_df.chrom == chrom, ['mid', 'start']].values)

bin_end_dict = collections.defaultdict(lambda: {})
for chrom in mycooler.chromnames:
    bin_end_dict[chrom] = dict(bin_df.loc[bin_df.chrom == chrom, ['mid', 'end']].values)

sig_df['start1'] = sig_df.apply(lambda row: bin_start_dict[row['chr1']][row['fragmentMid1']], axis=1)
sig_df['start2'] = sig_df.apply(lambda row: bin_start_dict[row['chr2']][row['fragmentMid2']], axis=1)

sig_df['name'] = sig_df.index
sig_df['strand'] = '.'
sig_df['score'] = -np.log10(sig_df['p-value'])
sig_df.loc[sig_df.score == np.inf, 'score'] = 1000

chr_matrix_list = []
for chrom in mycooler.chromnames:
    chr_matrix = mycooler.matrix(balance=True, as_pixels=True, join=True).fetch(chrom).fillna(0)
    chr_matrix = chr_matrix[(abs(chr_matrix.start1 - chr_matrix.start2) >= 20000) & (abs(chr_matrix.start1 - chr_matrix.start2) <= 2000000)]
    chr_matrix_list.append(chr_matrix)

matrix_df = pd.concat(chr_matrix_list).set_index(['chrom1', 'start1', 'start2'])
sig_df['balanced_count'] = sig_df.apply(lambda row: matrix_df.loc[idx[row['chr1'], row['start1'], row['start2']], 'balanced'], axis=1)

stim_DKO_sig_df = sig_df

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log10


In [12]:
up_gene_WT_interactions_df_list = []

score_cutoff = 0
cutoff_sig_df = WT_sig_df[WT_sig_df.score >= score_cutoff]
cutoff_sig_1_df = cutoff_sig_df.set_index(['chr1', 'start1', 'start2'])
cutoff_sig_2_df = cutoff_sig_df.set_index(['chr1', 'start2', 'start1'])
idx = pd.IndexSlice

for gene_id, gene_iv in up_genes_dict.iteritems():
    chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    try:
        gene_cutoff_sig_2_df = cutoff_sig_2_df.loc[idx[chrom, gene_bin_start],['score', 'balanced_count']]
        gene_cutoff_sig_2_df['interaction_site_bin_start'] = gene_cutoff_sig_2_df.index
        gene_cutoff_sig_2_df['gene_id'] = gene_id
        gene_cutoff_sig_2_df['gene_TSS_bin_start'] = gene_bin_start
        gene_cutoff_sig_2_df['chrom'] = chrom
    except:
        gene_cutoff_sig_2_df = None
    
    try:
        gene_cutoff_sig_1_df = cutoff_sig_1_df.loc[idx[chrom, gene_bin_start],['score', 'balanced_count']]
        gene_cutoff_sig_1_df['interaction_site_bin_start'] = gene_cutoff_sig_1_df.index
        gene_cutoff_sig_1_df['gene_id'] = gene_id
        gene_cutoff_sig_1_df['gene_TSS_bin_start'] = gene_bin_start
        gene_cutoff_sig_1_df['chrom'] = chrom
    except:
        gene_cutoff_sig_1_df = None
    
    if gene_cutoff_sig_1_df is not None or gene_cutoff_sig_2_df is not None:
        up_gene_WT_interactions_df_list.append(pd.concat([gene_cutoff_sig_1_df, gene_cutoff_sig_2_df])) 

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:1310: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [14]:
up_gene_stim_DKO_interactions_df_list = []

score_cutoff = 0
cutoff_sig_df = stim_DKO_sig_df[stim_DKO_sig_df.score >= score_cutoff]
cutoff_sig_1_df = cutoff_sig_df.set_index(['chr1', 'start1', 'start2'])
cutoff_sig_2_df = cutoff_sig_df.set_index(['chr1', 'start2', 'start1'])
idx = pd.IndexSlice

for gene_id, gene_iv in up_genes_dict.iteritems():
    chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    try:
        gene_cutoff_sig_2_df = cutoff_sig_2_df.loc[idx[chrom, gene_bin_start],['score', 'balanced_count']]
        gene_cutoff_sig_2_df['interaction_site_bin_start'] = gene_cutoff_sig_2_df.index
        gene_cutoff_sig_2_df['gene_id'] = gene_id
        gene_cutoff_sig_2_df['gene_TSS_bin_start'] = gene_bin_start
        gene_cutoff_sig_2_df['chrom'] = chrom
    except:
        gene_cutoff_sig_2_df = None
    
    try:
        gene_cutoff_sig_1_df = cutoff_sig_1_df.loc[idx[chrom, gene_bin_start],['score', 'balanced_count']]
        gene_cutoff_sig_1_df['interaction_site_bin_start'] = gene_cutoff_sig_1_df.index
        gene_cutoff_sig_1_df['gene_id'] = gene_id
        gene_cutoff_sig_1_df['gene_TSS_bin_start'] = gene_bin_start
        gene_cutoff_sig_1_df['chrom'] = chrom
    except:
        gene_cutoff_sig_1_df = None
        
    if gene_cutoff_sig_1_df is not None or gene_cutoff_sig_2_df is not None:
        up_gene_stim_DKO_interactions_df_list.append(pd.concat([gene_cutoff_sig_1_df, gene_cutoff_sig_2_df]))

In [15]:
peak_chrom_dict = collections.defaultdict(lambda: [])
for peak, peak_iv in  peaks_dict.iteritems():
    peak_mid = peak_iv.start
    peak_bin_start = peak_mid / 10000 * 10000
    chrom = peak_iv.chrom
    peak_chrom_dict[chrom].append(peak_bin_start)

In [16]:
up_gene_WT_interactions_df = pd.concat(up_gene_WT_interactions_df_list).loc[:,['gene_id', 'chrom', 'gene_TSS_bin_start', 'interaction_site_bin_start', 'score', 'balanced_count']]
up_gene_WT_interactions_df['Tcf1_binding_at_gene_TSS'] = up_gene_WT_interactions_df.apply(lambda row: row['gene_TSS_bin_start'] in peak_chrom_dict[row['chrom']], axis=1)
up_gene_WT_interactions_df['Tcf1_binding_at_interaction_site'] = up_gene_WT_interactions_df.apply(lambda row: row['interaction_site_bin_start'] in peak_chrom_dict[row['chrom']], axis=1)

In [17]:
up_gene_stim_DKO_interactions_df = pd.concat(up_gene_stim_DKO_interactions_df_list).loc[:,['gene_id', 'chrom', 'gene_TSS_bin_start', 'interaction_site_bin_start', 'score', 'balanced_count']]
up_gene_stim_DKO_interactions_df['Tcf1_binding_at_gene_TSS'] = up_gene_stim_DKO_interactions_df.apply(lambda row: row['gene_TSS_bin_start'] in peak_chrom_dict[row['chrom']], axis=1)
up_gene_stim_DKO_interactions_df['Tcf1_binding_at_interaction_site'] = up_gene_stim_DKO_interactions_df.apply(lambda row: row['interaction_site_bin_start'] in peak_chrom_dict[row['chrom']], axis=1)

In [18]:
up_gene_lost_interactions_df = up_gene_WT_interactions_df[up_gene_WT_interactions_df.apply(lambda row: row['interaction_site_bin_start'] not in up_gene_stim_DKO_interactions_df.loc[up_gene_stim_DKO_interactions_df.gene_id == row['gene_id'], 'interaction_site_bin_start'].tolist(), axis=1)]
up_gene_lost_interactions_df.rename(columns={'score': 'WT_CD8_score', 'balanced_count': 'WT_CD8_balanced_count'}, inplace=True)

/usr/local/lib/python2.7/dist-packages/pandas/core/frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


## prep

In [22]:
sig_df = pd.read_csv('fit-hi-c/stim_DKO_CD8.spline_pass2.significances.txt.gz', header=0, sep='\t')

mycooler = cooler.Cooler('stim_DKO_CD8-res-10K.cool')
bin_df = mycooler.bins()[:]
bin_df['mid'] = ((bin_df['start'] + bin_df['end']) / 2).astype(int)

bin_start_dict = collections.defaultdict(lambda: {})
for chrom in mycooler.chromnames:
    bin_start_dict[chrom] = dict(bin_df.loc[bin_df.chrom == chrom, ['mid', 'start']].values)

bin_end_dict = collections.defaultdict(lambda: {})
for chrom in mycooler.chromnames:
    bin_end_dict[chrom] = dict(bin_df.loc[bin_df.chrom == chrom, ['mid', 'end']].values)

sig_df['start1'] = sig_df.apply(lambda row: bin_start_dict[row['chr1']][row['fragmentMid1']], axis=1)
sig_df['start2'] = sig_df.apply(lambda row: bin_start_dict[row['chr2']][row['fragmentMid2']], axis=1)

sig_df['name'] = sig_df.index
sig_df['strand'] = '.'
sig_df['score'] = -np.log10(sig_df['p-value'])
sig_df.loc[sig_df.score == np.inf, 'score'] = 1000

chr_matrix_list = []
for chrom in mycooler.chromnames:
    chr_matrix = mycooler.matrix(balance=True, as_pixels=True, join=True).fetch(chrom).fillna(0)
    chr_matrix = chr_matrix[(abs(chr_matrix.start1 - chr_matrix.start2) >= 20000) & (abs(chr_matrix.start1 - chr_matrix.start2) <= 2000000)]
    chr_matrix_list.append(chr_matrix)

matrix_df = pd.concat(chr_matrix_list).set_index(['chrom1', 'start1', 'start2'])
sig_df['balanced_count'] = sig_df.apply(lambda row: matrix_df.loc[idx[row['chr1'], row['start1'], row['start2']], 'balanced'], axis=1)

stim_DKO_all_df = sig_df

stim_DKO_all_df.set_index(['chr1', 'start1', 'start2'], inplace=True)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log10


## consider from Tcf1 peaks perspective

In [19]:
Tcf1_WT_interactions_df_list = []
Tcf1_stim_DKO_interactions_df_list = []

score_cutoff = 0
cutoff_sig_df = WT_sig_df[WT_sig_df.score >= score_cutoff]
cutoff_sig_1_df = cutoff_sig_df.set_index(['chr1', 'start1', 'start2'])
cutoff_sig_2_df = cutoff_sig_df.set_index(['chr1', 'start2', 'start1'])
idx = pd.IndexSlice

for peak, peak_iv in peaks_dict.iteritems():
    peak_mid = peak_iv.start
    peak_bin_start = peak_mid / 10000 * 10000
    chrom = peak_iv.chrom
    
    try:
        peak_cutoff_sig_2_df = cutoff_sig_2_df.loc[idx[chrom, peak_bin_start],['score', 'balanced_count']]
        peak_cutoff_sig_2_df['interaction_site_bin_start'] = peak_cutoff_sig_2_df.index
        peak_cutoff_sig_2_df['Tcf1_binding_bin_start'] = peak_bin_start
        peak_cutoff_sig_2_df['chrom'] = chrom
        peak_cutoff_sig_2_df['Tcf1_binding_iv'] = peak
    except:
        peak_cutoff_sig_2_df = None
    
    try:
        peak_cutoff_sig_1_df = cutoff_sig_1_df.loc[idx[chrom, peak_bin_start],['score', 'balanced_count']]
        peak_cutoff_sig_1_df['interaction_site_bin_start'] = peak_cutoff_sig_1_df.index
        peak_cutoff_sig_1_df['Tcf1_binding_bin_start'] = peak_bin_start
        peak_cutoff_sig_1_df['chrom'] = chrom
        peak_cutoff_sig_1_df['Tcf1_binding_iv'] = peak
    except:
        peak_cutoff_sig_1_df = None
    
    if peak_cutoff_sig_1_df is not None or peak_cutoff_sig_2_df is not None:
        Tcf1_WT_interactions_df_list.append(pd.concat([peak_cutoff_sig_1_df, peak_cutoff_sig_2_df])) 
        
        
score_cutoff = 0
cutoff_sig_df = stim_DKO_sig_df[stim_DKO_sig_df.score >= score_cutoff]
cutoff_sig_1_df = cutoff_sig_df.set_index(['chr1', 'start1', 'start2'])
cutoff_sig_2_df = cutoff_sig_df.set_index(['chr1', 'start2', 'start1'])
idx = pd.IndexSlice

for peak, peak_iv in peaks_dict.iteritems():
    peak_mid = peak_iv.start
    peak_bin_start = peak_mid / 10000 * 10000
    chrom = peak_iv.chrom
    
    try:
        peak_cutoff_sig_2_df = cutoff_sig_2_df.loc[idx[chrom, peak_bin_start],['score', 'balanced_count']]
        peak_cutoff_sig_2_df['interaction_site_bin_start'] = peak_cutoff_sig_2_df.index
        peak_cutoff_sig_2_df['Tcf1_binding_bin_start'] = peak_bin_start
        peak_cutoff_sig_2_df['chrom'] = chrom
        peak_cutoff_sig_2_df['Tcf1_binding_iv'] = peak
    except:
        peak_cutoff_sig_2_df = None
    
    try:
        peak_cutoff_sig_1_df = cutoff_sig_1_df.loc[idx[chrom, peak_bin_start],['score', 'balanced_count']]
        peak_cutoff_sig_1_df['interaction_site_bin_start'] = peak_cutoff_sig_1_df.index
        peak_cutoff_sig_1_df['Tcf1_binding_bin_start'] = peak_bin_start
        peak_cutoff_sig_1_df['chrom'] = chrom
        peak_cutoff_sig_1_df['Tcf1_binding_iv'] = peak
    except:
        peak_cutoff_sig_1_df = None
    
    if peak_cutoff_sig_1_df is not None or peak_cutoff_sig_2_df is not None:
        Tcf1_stim_DKO_interactions_df_list.append(pd.concat([peak_cutoff_sig_1_df, peak_cutoff_sig_2_df])) 

In [20]:
Tcf1_WT_interactions_df = pd.concat(Tcf1_WT_interactions_df_list).loc[:,['Tcf1_binding_iv', 'chrom', 'Tcf1_binding_bin_start', 'interaction_site_bin_start', 'score', 'balanced_count']]
Tcf1_WT_interactions_df.rename(columns={'score': 'WT_CD8_score', 'balanced_count': 'WT_CD8_balanced_count'}, inplace=True)
#Tcf1_WT_interactions_df.set_index(['chrom', 'Tcf1_binding_bin_start', 'interaction_site_bin_start'], inplace=True)

In [21]:
Tcf1_stim_DKO_interactions_df = pd.concat(Tcf1_stim_DKO_interactions_df_list).loc[:,['Tcf1_binding_iv', 'chrom', 'Tcf1_binding_bin_start', 'interaction_site_bin_start', 'score', 'balanced_count']]
Tcf1_stim_DKO_interactions_df.rename(columns={'score': 'stim_DKO_CD8_score', 'balanced_count': 'stim_DKO_CD8_balanced_count'}, inplace=True)

In [23]:
Tcf1_CD8_interactions_df = Tcf1_WT_interactions_df.merge(Tcf1_stim_DKO_interactions_df, on=['Tcf1_binding_iv', 'chrom', 'Tcf1_binding_bin_start', 'interaction_site_bin_start'], how='left')

def retrieve_interaction_score_from_stim_DKO(row, colname):
    start1 = min(row['Tcf1_binding_bin_start'], row['interaction_site_bin_start'])
    start2 = max(row['Tcf1_binding_bin_start'], row['interaction_site_bin_start'])
    try:
        return stim_DKO_all_df.loc[idx[row['chrom'], start1, start2], colname]
    except:
        return 0

Tcf1_CD8_interactions_df['stim_DKO_CD8_score'] = Tcf1_CD8_interactions_df.apply(lambda row: retrieve_interaction_score_from_stim_DKO(row, 'score'), axis=1)
Tcf1_CD8_interactions_df['stim_DKO_CD8_balanced_count'] = Tcf1_CD8_interactions_df.apply(lambda row: retrieve_interaction_score_from_stim_DKO(row, 'balanced_count'), axis=1)

In [24]:
Tcf1_CD8_interactions_df['diff_score'] = Tcf1_CD8_interactions_df['WT_CD8_score'] - Tcf1_CD8_interactions_df['stim_DKO_CD8_score']
Tcf1_CD8_interactions_df['diff_balanced_count'] = Tcf1_CD8_interactions_df['WT_CD8_balanced_count'] - Tcf1_CD8_interactions_df['stim_DKO_CD8_balanced_count']

In [25]:
Tcf1_CD8_interactions_df.shape

(318063, 10)

In [34]:
Tcf1_CD8_interactions_df['diff_score'].median()

1.6427883613971659

In [26]:
Tcf1_CD8_interactions_df.to_csv('Tcf1_CD8_interactions.tsv', sep='\t', index=None)

In [ ]:
Tcf1_CD8_interactions_df = pd.read_csv('Tcf1_CD8_interactions.tsv', sep='\t', header=0)

In [27]:
all_genes_dict = {}

all_genes_bed_file = HTSeq.BED_Reader('/home/zzeng/cloud_research/PengGroup/ZZeng/Annotation/gene_iv/mm9/gene_promoter_1k_iv_unique.bed')
for alt in all_genes_bed_file:
    all_genes_dict[alt.name] = alt.iv

In [50]:
top_Tcf1_CD8_interactions_df = Tcf1_CD8_interactions_df.sort_values(by='diff_score', ascending=True).iloc[:200000,:]

In [29]:
all_genes_tuples = []

for gene_id, gene_iv in all_genes_dict.iteritems():
    gene_chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    all_genes_tuples.append((gene_chrom, gene_bin_start))
all_genes_tuples = set(all_genes_tuples)

up_genes_tuples = []

for gene_id, gene_iv in up_genes_dict.iteritems():
    gene_chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    up_genes_tuples.append((gene_chrom, gene_bin_start))
up_genes_tuples = set(up_genes_tuples)

down_genes_tuples = []

for gene_id, gene_iv in down_genes_dict.iteritems():
    gene_chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    down_genes_tuples.append((gene_chrom, gene_bin_start))
down_genes_tuples = set(down_genes_tuples)

In [30]:
all_genes_tuples_dict = collections.defaultdict(lambda: [])

for gene_id, gene_iv in all_genes_dict.iteritems():
    gene_chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    all_genes_tuples_dict[(gene_chrom, gene_bin_start)].append(gene_id)

up_genes_tuples_dict = collections.defaultdict(lambda: [])

for gene_id, gene_iv in up_genes_dict.iteritems():
    gene_chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    up_genes_tuples_dict[(gene_chrom, gene_bin_start)].append(gene_id)

down_genes_tuples_dict = collections.defaultdict(lambda: [])

for gene_id, gene_iv in down_genes_dict.iteritems():
    gene_chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    down_genes_tuples_dict[(gene_chrom, gene_bin_start)].append(gene_id)

In [51]:
top_Tcf1_CD8_interactions_df['all_gene_TSS_at_interaction_site'] = top_Tcf1_CD8_interactions_df.apply(lambda row: True if (row['chrom'], row['interaction_site_bin_start']) in all_genes_tuples else False, axis=1)
top_Tcf1_CD8_interactions_df['up_gene_TSS_at_interaction_site'] = top_Tcf1_CD8_interactions_df.apply(lambda row: True if (row['chrom'], row['interaction_site_bin_start']) in up_genes_tuples else False, axis=1)
top_Tcf1_CD8_interactions_df['down_gene_TSS_at_interaction_site'] = top_Tcf1_CD8_interactions_df.apply(lambda row: True if (row['chrom'], row['interaction_site_bin_start']) in down_genes_tuples else False, axis=1)

In [52]:
print len(all_genes_tuples), top_Tcf1_CD8_interactions_df[top_Tcf1_CD8_interactions_df.all_gene_TSS_at_interaction_site].shape[0]
print len(up_genes_tuples), top_Tcf1_CD8_interactions_df[top_Tcf1_CD8_interactions_df.up_gene_TSS_at_interaction_site].shape[0]
print len(down_genes_tuples), top_Tcf1_CD8_interactions_df[top_Tcf1_CD8_interactions_df.down_gene_TSS_at_interaction_site].shape[0]

20910 50977
296 1229
151 585


In [53]:
print len(set(reduce(lambda x, y: x + y, top_Tcf1_CD8_interactions_df[top_Tcf1_CD8_interactions_df.all_gene_TSS_at_interaction_site].apply(lambda row: all_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), [])))
print len(set(reduce(lambda x, y: x + y, top_Tcf1_CD8_interactions_df[top_Tcf1_CD8_interactions_df.up_gene_TSS_at_interaction_site].apply(lambda row: up_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), [])))
print len(set(reduce(lambda x, y: x + y, top_Tcf1_CD8_interactions_df[top_Tcf1_CD8_interactions_df.down_gene_TSS_at_interaction_site].apply(lambda row: down_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), [])))

13374
255
120


In [43]:
print set(reduce(lambda x, y: x + y, top_Tcf1_CD8_interactions_df[top_Tcf1_CD8_interactions_df.up_gene_TSS_at_interaction_site].apply(lambda row: up_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), []))
print set(reduce(lambda x, y: x + y, top_Tcf1_CD8_interactions_df[top_Tcf1_CD8_interactions_df.down_gene_TSS_at_interaction_site].apply(lambda row: down_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), []))

set(['Recql4', 'Tha1', 'Pola2', 'Pim1', 'Med20', 'Map3k8', 'Tfrc', 'Ctc1', 'Ms4a4c', 'Fbxl18', 'Cbx1', 'Slc25a10'])
set(['Orai1', 'Ercc5', 'Ap3m2', 'Snord83b', 'Amd2', 'Suco'])


In [44]:
', '.join(['Recql4', 'Tha1', 'Pola2', 'Pim1', 'Med20', 'Map3k8', 'Tfrc', 'Ctc1', 'Ms4a4c', 'Fbxl18', 'Cbx1', 'Slc25a10'])

'Recql4, Tha1, Pola2, Pim1, Med20, Map3k8, Tfrc, Ctc1, Ms4a4c, Fbxl18, Cbx1, Slc25a10'

In [45]:
', '.join(['Orai1', 'Ercc5', 'Ap3m2', 'Snord83b', 'Amd2', 'Suco'])

'Orai1, Ercc5, Ap3m2, Snord83b, Amd2, Suco'

In [55]:
import scipy

In [71]:
scipy.stats.chi2_contingency(np.array([[20910, 151], [13374, 120]]))

(2.9210525832220506,
 0.087430814456609718,
 1,
 array([[ 20895.8276371,    165.1723629],
        [ 13388.1723629,    105.8276371]]))